# Convolution

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
import cv2

In [2]:
import tensorflow as tf

### Reading data as image and resizing and test, train & validation separated

In [11]:
def read_data(path):
    
    train_path = path+"/train"
    test_path = path+"/test"
    validation_path = path+"/val"
    
    tr_data, test_data, val_data = [], [], []
    tr_out, test_out, val_out = [], [], []

    for i in os.listdir(train_path):

        try:
            for j in os.listdir(train_path+"/"+i):
                image = cv2.imread(train_path+"/"+i+"/"+j, cv2.IMREAD_GRAYSCALE)
                tr_data.append(cv2.resize(image, (224, 224)))
                tr_out.append(i)

            for j in os.listdir(test_path+"/"+i):
                image = cv2.imread(test_path+"/"+i+"/"+j, cv2.IMREAD_GRAYSCALE)
                test_data.append(cv2.resize(image, (224, 224)))
                test_out.append(i)

            for j in os.listdir(validation_path+"/"+i):
                image = cv2.imread(validation_path+"/"+i+"/"+j, cv2.IMREAD_GRAYSCALE)
                val_data.append(cv2.resize(image, (224, 224)))
                val_out.append(i)
        except:
            pass
                
                
    tr_data, test_data, val_data = np.array(tr_data), np.array(test_data), np.array(val_data)
    tr_out, test_out, val_out = np.array(tr_out), np.array(test_out), np.array(val_out)

    return tr_data, test_data, val_data, tr_out, test_out, val_out



In [12]:
path = "/Users/rajeshr/Desktop/Group_22"
tr_data, test_data, val_data, tr_out, test_out, val_out = read_data(path)

## Kaiming Initializatied Filter

In [167]:
def kaiming(n, size):
    mean, sd = 0, np.sqrt(2/n)
    weights = np.random.normal(mean, sd, size=size)
    return weights

In [391]:
np.random.seed(0)
n = 1
filtr = kaiming(n, (3, 3))
filtr

array([[ 2.49474675,  0.56590775,  1.38414453],
       [ 3.16910155,  2.64112584, -1.38207963],
       [ 1.34362793, -0.21405142, -0.1459735 ]])

## Convolutional Filter

In [390]:
def zero_padding(inpt, padding):
    
    m, n = inpt.shape
    
    temp = []
    for i in range(padding):
        for j in range(m+2*padding):
            temp.append(0)
            
    for i in inpt:
        
        for j in range(padding):
            temp.append(0)
        
        for p in i:
            temp.append(p)
            
        for k in range(padding):
            temp.append(0)
            
    for i in range(padding):
        for j in range(m+2*padding):
            temp.append(0)
    
    
    temp = np.array(temp, dtype=np.uint8)
    temp = temp.flatten()
    temp = temp.reshape(m+2*padding, n+2*padding)
    
    return temp

In [399]:
def convolution(inpt, filtr , stride, padding):
    
    inpt = zero_padding(inpt, padding)
    
    (m, n), (fm, fn) = inpt.shape, filtr.shape
    
    feature_map = []
    
    flip_leftright = np.fliplr(filtr)
    flip_updown = np.flipud(flip_leftright)
    Filtr = flip_updown 
    
    try:
        feature_map = np.zeros((int(((m-fm+2*padding)/stride)+1), int(((n-fn+2*padding)/stride)+1)))
    except:
        print("Stride can't be ZERO (Stride should be ℕ)\nWILL GENERATE ERRORS")
    
    print(feature_map.shape)
    
    for i in range(n):
        if i <= n - fn:
            if i%stride == 0:
                for j in range(m):
                    if j <= m - fm:
                        if j % stride == 0:
                            try:
                                feature_map[j, i] = (Filtr*inpt[j:j+fm, i:i+fn]).sum()
                            except:
                                break
                        
                    
    
    return feature_map

feature_map = convolution(tr_data[0], filtr, stride = 2, padding = 1)
print(feature_map)
cv2.imwrite('/Users/rajeshr/Desktop/feature_map'+str(np.random.randint(1000))+'.jpg', feature_map)
tr_data[0].shape, feature_map.shape

(113, 113)
[[  88.147013      0.           98.18254414 ...  325.11711373
     0.          433.73105887]
 [   0.            0.            0.         ...    0.
     0.            0.        ]
 [ 137.06247287    0.          188.29690382 ...  307.52215966
     0.          407.83340266]
 ...
 [ 227.08523347    0.          228.62095747 ... 1529.38256994
     0.         1376.68240458]
 [   0.            0.            0.         ...    0.
     0.            0.        ]
 [ 222.88165187    0.          225.34471918 ... 1384.81369991
     0.         1320.94986305]]


((224, 224), (113, 113))

In [436]:
import tensorflow as tf

def comp_conv2d(conv2d, X):
  
    X = tf.reshape(X, (1, ) + X.shape + (1, ))
    Y = conv2d(X)

    return tf.reshape(Y, Y.shape[1:3])

conv2d = tf.keras.layers.Conv2D(1, kernel_size=3, padding='same', strides=2)
inputs = np.array(tr_data[0], dtype=np.float)
k = comp_conv2d(conv2d, inputs)
k = np.array(k)
cv2.imwrite('/Users/rajeshr/Desktop/k'+str(np.random.randint(1000))+'.jpg', k)
k.shape

/var/folders/3t/vg36gb2936s7j53bxphjbg1c0000gn/T/ipykernel_51338/3954544478.py:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  inputs = np.array(tr_data[0], dtype=np.float)


(112, 112)